# BERTopic 


источник всего кода: https://github.com/MaartenGr/BERTopic/blob/master/notebooks/BERTopic.ipynb


версия с учителем: https://colab.research.google.com/drive/1bxizKzv5vfxJEB29sntU__ZC7PBSIPaQ?usp=sharing

inspired by bertopic_agro_ex (created by Marina)

## Выгрузка данных

In [1]:
# SPECIAL PACKAGES FOR PRELIMINARY ANALYSIS

import os
import numpy as np
import pandas as pd
import torch
import re
import nltk
import spacy
import string
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None

from pathlib import Path

In [15]:
RAW_DATA_PATH = Path().absolute().parent / 'raw_data'
PROCESSED_DATA_PATH = Path().absolute().parent / 'processed_data'
TEMP_RESULTS_PATH = Path().absolute().parent / 'intermediate_results'

TEMP_RESULTS_PATH = Path(r'C:\Users\EPostolit\Programs\vega\while disk failures\topic_modelling\intermediate_results')

MODELS_PATH = TEMP_RESULTS_PATH / 'tm_models'
TM_DYN_PATH = TEMP_RESULTS_PATH / 'tm_dynamics'
TM_PROBS_PATH = TEMP_RESULTS_PATH / 'tm_probs'
TM_FIGURES_PATH = TEMP_RESULTS_PATH / 'figures'


for fold in [MODELS_PATH, TM_DYN_PATH, TM_PROBS_PATH, TM_FIGURES_PATH]:
    if not os.path.exists(fold):
        os.makedirs(fold)


DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE

'cuda'

In [3]:
import plotly.graph_objects as go
import typing as tp

from copy import deepcopy


EXPORT_PATH = TM_FIGURES_PATH

def export_figure(fig: go.Figure, name: str, voc=None,
                  use_default_path: bool = True,
                  save_types: tp.Tuple[str] = ('png', 'html', 'svg'),
                  tg_only = True,
                  scale: int = 4) -> None:
    """
    :param go.Figure fig: figure to export
    :param str name: name of a figure (without path prefix and file extention)
    :param voc: transation vocabulary
    :param bool use_default_path: whether to add a path prefix to the name
    :param tp.Tuple[str] save_types: figure extention to save
    :param int scale: scale factore (in range (1, 7)), the higher, the better quality
    """
    def _save_figure(fig, name, ext):
        path = EXPORT_PATH / name if use_default_path else name
        path = str(path) + '.' + ext
        if ext == 'json':
            with open(path, 'w', encoding='utf-8') as file:
                fig.write_json(path, file)
        if ext != 'html':
            fig.write_image(path, scale=scale)
        else:
            fig.write_html(path)

    fig_names_to_save = [(fig, name)]

    for ext in save_types:
        for fig, n in fig_names_to_save:
            _save_figure(fig, n, ext)

In [4]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

### [пропускаем] следующие ячейки, чтобы достать данные из pkl ⛹

In [5]:
# !pip install google-colab
# from google.colab import files
# upload = files.upload()

In [6]:
import pickle

with open(RAW_DATA_PATH / 'messages_many.pkl', 'rb') as f:
    data_df = pickle.load(f)

data_df.sort_values(by='DatePosted', inplace=True)
data_df['MessageText'] = data_df['MessageText'].astype(str)
data_df

,MessageID,ChannelID,DateAdded,DatePosted,MessageText,IsForward
0,1,1006112008,2021-02-06 00:53:10,2015-11-04 08:50:42,,False
1,5,1006112008,2021-02-06 00:53:10,2015-11-05 09:08:08,,False
2,6,1006112008,2021-02-06 00:53:10,2015-11-05 11:20:13,@economika - новости экономики кратко,False
3,10,1006112008,2021-02-06 00:53:10,2015-11-05 11:28:24,Авиакомпания Победа в ближайшие дни начинает п...,False
4,12,1006112008,2021-02-06 00:53:10,2015-11-05 17:50:24,Реальная заработная плата в этом году снизится...,False
...,...,...,...,...,...,...
1214676,2388,1734623470,2023-05-25 13:26:18,2023-05-25 13:11:44,"""Итоги месяца маркировки пива. ЦРПТ поделился ...",False
1215234,568,1868097154,2023-05-25 13:26:20,2023-05-25 13:16:19,"""Вагоны идут под откос. В Ленобласти открыли ...",False
1212376,577,1679099655,2023-05-25 13:26:16,2023-05-25 13:18:34,"""""""Европейский рыбный центр"""" построит порт во...",False
1202643,8431,1464785872,2023-05-25 13:20:52,2023-05-25 13:20:21,,False


In [7]:
chn_info_df = pd.read_excel(RAW_DATA_PATH / 'channels.xlsx')
chn_info_df.head()

,ChannelID,ChannelName,ChannelURL,DateAdded,IsCrawling,ChannelTypeCode
0,1006112008,Экономика,https://t.me/economika,2021-02-02 17:35:23,True,5
1,1009962628,TechSparks,https://t.me/techsparks,2021-02-02 17:35:28,True,5
2,1036240821,Медуза,https://t.me/meduzalive,2021-02-06 00:52:00,True,4
3,1038402501,Коммерсант,https://t.me/kommersant,2021-02-06 01:15:48,True,4
4,1075565753,Ведомости,https://t.me/vedomosti,2021-02-06 01:03:13,True,4


In [8]:
chn_id2name = dict(zip(chn_info_df['ChannelID'], chn_info_df['ChannelName']))

### теперь данные из одного источника (AGRO)

In [9]:
# !pip install google-colab
# from google.colab import filesb
# upload = files.upload()

In [10]:
#import telegram-data from one source for 10 000 tweets
# papers = pd.read_excel('data_Agro.xlsx')

#read the values of the file in the dataframe
# data = pd.DataFrame(papers, columns=['MessageID', 'ChannelID', 'DateAdded', 'DatePosted', 'MessageText', 'IsForward'])

#change table, remove the columns
# papers = papers.drop(columns=['MessageID', 'ChannelID', 'DateAdded', 'IsForward'], axis=1)


# papers.head()

# BERT MODELLING

We can specialize the type of embeddings-modelling, the parameter n for n-grams, the minimum number of words in one topic 

https://colab.research.google.com/drive/1ClTYut039t-LDtlcd-oQAdXWgcsSGTw9?usp=sharing

In [11]:
# !pip install -U sentence-transformers
# from sentence_transformers import SentenceTransformer
# embedding_model = SentenceTransformer('all-mpnet-base-v2')

# !pip install umap-learn
# from umap import UMAP
# umap_model = UMAP(n_neighbors=15)

# !pip install hdbscan
# from hdbscan import HDBSCAN
# hdbscan_model = HDBSCAN(min_cluster_size=20, min_samples=1,
                        # gen_min_span_tree=True)


### [пропускаем] если надо включить стоп-слова, можно выполнить следующие ячейки

In [12]:
# import nltk
# nltk.download("stopwords")

# from nltk.corpus import stopwords
# ", ".join(stopwords.words('russian'))
# STOPWORDS = set(stopwords.words('russian'))
# STOPWORDS = stopwords.words('russian')
# STOPWORDS.extend(["иностранного", "из-за", "иностранными", "иностранным", "агента", "функции", "выполняющим", "создано", "https", "http", "kommersant", 
#                    "reuters", "reuters_russia", "utm_media", "utm_campaign", "telegram",
#                   "www.kommersant.ru", "vdmsti.ru", "vedomosti", "mdzaio", "meduza.io", "рейтер", "информацию", "данное", "информацию", "сообщение", "массовом",
#                   "информации", "материал", "сообщение", "распространено", "млн", "млрд", "объемы", "лицом", "глава",
#                   "live", "также", "это", "сообщил", "агенство", "tco", "nan", "cbonds", "средством", "россииским",
#                    "массовой", "ampgs", "ytm", "eff", "года", "сутки", "january", "february", "march", "april", "may",
#                   "june", "july", "august", "september", "october", "september", "november", "december", "doc", "который", "которую", 
#                    "которой", "которые", "который", "будут", "news", "против", "массовый"])

# from sklearn.feature_extraction.text import CountVectorizer
# vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words=STOPWORDS)

In [13]:
un_chan = np.unique(data_df['ChannelID'])
channel_id = un_chan[0]
channel_df = data_df[data_df['ChannelID'] == channel_id]
channel_df.sort_values(by='DatePosted', inplace=True)
channel_df

,MessageID,ChannelID,DateAdded,DatePosted,MessageText,IsForward
0,1,1006112008,2021-02-06 00:53:10,2015-11-04 08:50:42,,False
1,5,1006112008,2021-02-06 00:53:10,2015-11-05 09:08:08,,False
2,6,1006112008,2021-02-06 00:53:10,2015-11-05 11:20:13,@economika - новости экономики кратко,False
3,10,1006112008,2021-02-06 00:53:10,2015-11-05 11:28:24,Авиакомпания Победа в ближайшие дни начинает п...,False
4,12,1006112008,2021-02-06 00:53:10,2015-11-05 17:50:24,Реальная заработная плата в этом году снизится...,False
...,...,...,...,...,...,...
28658,29624,1006112008,2023-05-25 12:56:27,2023-05-25 08:04:03,"Банк России продлил еще на шесть месяцев, до к...",False
28659,29625,1006112008,2023-05-25 12:56:27,2023-05-25 09:02:04,В первом квартале 2023 года инвесторы нарастил...,False
28660,29626,1006112008,2023-05-25 12:56:27,2023-05-25 10:01:04,В Госдуме предложили обязать школьников трудит...,False
28661,29627,1006112008,2023-05-25 12:56:27,2023-05-25 11:44:16,,True


## непосредственно модель


In [43]:
from bertopic import BERTopic
from tqdm.auto import tqdm


for i, channel_id in tqdm(enumerate(np.unique(data_df['ChannelID'])),
                          total = len(np.unique(data_df['ChannelID']))):
    # if i < 20:  # in case some channels already processed
        # continue

    channel_df = data_df[data_df['ChannelID'] == channel_id]
    channel_df.sort_values(by='DatePosted', inplace=True)
    channel_df.reset_index(drop=True, inplace=True)
    
    model = BERTopic(
        language='russian',
        calculate_probabilities=True,
        verbose=True
    )

    topics, probs = model.fit_transform(channel_df['MessageText'])

    channel_df['topics_predicted'] = topics
    channel_df['probs_predicted'] = [p for p in probs]
    
    with open(TM_PROBS_PATH / f'{channel_id}_probs.pkl', 'wb') as f:
        pickle.dump(channel_df, f)

    channel_df['DateDay'] = channel_df['DatePosted'].apply(lambda d: pd.to_datetime(d.strftime('%Y-%m-%d')))
    days = (max(channel_df['DatePosted']) - min(channel_df['DatePosted'])).days
    tot = model.topics_over_time(channel_df['MessageText'],
                       channel_df['DateDay'],
                       global_tuning=True,
                       evolution_tuning=True,
                       nr_bins=days + 1)
    
    tot.to_excel(TM_DYN_PATH / f'{channel_id}_dynamic_topics.xlsx', index=False)
    
    fig = model.visualize_hierarchy(top_n_topics=50)
    export_figure(fig, f'{channel_id}_hierarchical clustering', scale=2)

    model.save(str(MODELS_PATH / f'bertopic_tmmodel_default_{channel_id}'))

  0%|          | 0/48 [00:00<?, ?it/s]

Batches:   0%|          | 0/7459 [00:00<?, ?it/s]

2023-05-29 15:22:34,093 - BERTopic - Transformed documents to Embeddings
2023-05-29 15:32:35,281 - BERTopic - Reduced dimensionality


In [ ]:
print(channel_id)

for i, chn_i in enumerate(np.unique(data_df['ChannelID'])):
    if i < 20:
        continue
    
    if i >= 20:
        print(chn_i)

1203560567

In [12]:
# BERToipc-modelling

# ! pip install bertopic

model = BERTopic(
    language='russian',
    calculate_probabilities=True,
    verbose=True
)

topics, probs = model.fit_transform(channel_df['MessageText'])

# We can then extract most frequent topics, -1 refers to all outliers and should typically be ignored
model.get_topic_freq().head(5)

Batches:   0%|          | 0/891 [00:00<?, ?it/s]

2023-05-25 12:15:17,545 - BERTopic - Transformed documents to Embeddings
2023-05-25 12:15:44,524 - BERTopic - Reduced dimensionality
2023-05-25 12:17:56,600 - BERTopic - Clustered reduced embeddings


,Topic,Count
0,-1,11297
1,0,602
2,1,579
3,2,520
4,3,422


In [19]:
model.get_topic_freq()

,Topic,Count
0,-1,11297
1,0,602
2,1,579
3,2,520
4,3,422
...,...,...
334,333,10
335,334,10
336,335,10
337,336,10


In [27]:
import sys

sys.getsizeof(model)

48

In [15]:
# Next, let's take a look at the most frequent topic that was generated
# Note that the model is stocastich which means that the topics might differ across runs.
model.get_topic(0)[:10]

[('вакцины', 0.015908339259077482),
 ('коронавируса', 0.014513203270489173),
 ('covid', 0.012136427962954215),
 ('19', 0.009592992378391908),
 ('вакцин', 0.009144987603113306),
 ('вакцинации', 0.008595308996442506),
 ('здравоохранения', 0.008461776893615217),
 ('спутник', 0.00734175061095012),
 ('коронавирусной', 0.0073337701299978604),
 ('инфекции', 0.007198233862407143)]

In [34]:
model.save('first_topic_model')

c:\Users\EPostolit\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\sparse\_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


### Attributes

In [16]:
# Access the predicted topics for the first 10 documents, we simply run the following:
model.topics_[:10]

[1, 1, 88, 144, -1, 1, 55, 18, -1, 20]

In [18]:
channel_df['topics_predicted'] = model.topics_
channel_df

,MessageID,ChannelID,DateAdded,DatePosted,MessageText,IsForward,topics_predicted
0,1,1006112008,2021-02-06 00:53:10,2015-11-04 08:50:42,,False,1
1,5,1006112008,2021-02-06 00:53:10,2015-11-05 09:08:08,,False,1
2,6,1006112008,2021-02-06 00:53:10,2015-11-05 11:20:13,@economika - новости экономики кратко,False,88
3,10,1006112008,2021-02-06 00:53:10,2015-11-05 11:28:24,Авиакомпания Победа в ближайшие дни начинает п...,False,144
4,12,1006112008,2021-02-06 00:53:10,2015-11-05 17:50:24,Реальная заработная плата в этом году снизится...,False,-1
...,...,...,...,...,...,...,...
28480,29446,1006112008,2023-05-06 18:55:52,2023-05-06 14:06:04,Следствие в рамках дела против инфоцыганки Еле...,False,9
28481,29447,1006112008,2023-05-06 18:55:52,2023-05-06 16:08:04,"Еврокомиссия считает, что у Евросоюза должна б...",False,-1
28482,29448,1006112008,2023-05-06 18:55:52,2023-05-06 18:07:03,Президент США Джо Байден призвал не дать выигр...,False,64
28483,29449,1006112008,2023-05-07 00:55:46,2023-05-06 19:12:52,"В Сбере рассказали, что выдали с момента начал...",False,-1


### Visualization


In [14]:
# We can visualize the topics that were generated in a way very similar to LDAvis (for saving time):
model.visualize_topics()

In [15]:
# visualize Topic Hierarchy

model.visualize_hierarchy(top_n_topics=50)

In [16]:
# visualize Terms

model.visualize_barchart(top_n_topics=20)

In [17]:
# visualize Topic Similarity

model.visualize_heatmap(n_clusters=20, width=1000, height=1000)

In [18]:
# visualize Term Score Decline

model.visualize_term_rank()

**комментарий к последней ячейке:**

*Темы представлены рядом слов, начинающихся с наиболее репрезентативного слова. Каждое слово представлено оценкой c-TF-IDF. Чем выше оценка, тем более репрезентативно слово для данной темы. Поскольку слова темы сортируются по их баллам c-TF-IDF, баллы медленно снижаются с каждым добавленным словом. В какой-то момент добавление слов к представлению темы лишь незначительно увеличивает общий балл c-TF-IDF и не принесет пользы для ее представления.*

*Чтобы визуализировать этот эффект, мы можем построить графики оценок c-TF-IDF по каждой теме по рангу термина каждого слова. Другими словами, позиция слов (ранг термина), где слова с наивысшим баллом c-TF-IDF будут иметь ранг 1, будет нанесена на ось x. Принимая во внимание, что ось y будет заполнена оценками c-TF-IDF. Результатом является визуализация, которая показывает вам снижение баллов c-TF-IDF при добавлении слов к представлению темы. Это позволяет вам, используя метод elbow, выбрать наилучшее количество слов в теме.*

### Search Topics

In [20]:
# After having trained our model, we can use find_topics to search for topics that are similar 
similar_topics, similarity = model.find_topics("Украина", top_n=5); similar_topics

[60, 51, 53, 40, 9]

In [21]:
model.get_topic(0)

[('молока', 0.0314561209002722),
 ('молоко', 0.013510176158554356),
 ('на', 0.01287883968064285),
 ('молочной', 0.012346880714702666),
 ('молочных', 0.010181145618602203),
 ('по', 0.00912719062578619),
 ('эконива', 0.009116517845900271),
 ('за', 0.008912962426400828),
 ('тонн', 0.008721703377828136),
 ('году', 0.008631164998534613)]

In [22]:
model.get_topic(3)

[('апк', 0.013275308172291491),
 ('для', 0.012501549983364455),
 ('от', 0.008585029054804514),
 ('технологии', 0.008467842229397504),
 ('по', 0.008376052913710557),
 ('это', 0.008067838298023114),
 ('на', 0.008023241663287797),
 ('данных', 0.007773048651559526),
 ('какие', 0.007485157960761025),
 ('россии', 0.00723244065201911)]

### Topics over Time

In [23]:
topics_over_time = model.topics_over_time(docs=papers['MessageText'], 
                                                timestamps=papers['DatePosted'], 
                                                global_tuning=True, 
                                                evolution_tuning=True, 
                                                nr_bins=20)

In [24]:
# Visualize Topics over Time
model.visualize_topics_over_time(topics_over_time, top_n_topics=20)

### Model saving

In [26]:
# Save model
model.save("my_model_AGRO")

In [28]:
# Load model
my_model = BERTopic.load("my_model_AGRO")

In [ ]:
topics_over_time.head

<bound method NDFrame.head of       Topic                                              Words  Frequency  \
0        -1                           mdza, io, https, или, не        174   
1         0     путин, нато, предложения, путина, безопасности          6   
2         1  js_azkofxhu, рекордное, седьмыми, баллистическ...          1   
3         2            dolce, gabbana, wildberries, крем, меха          6   
4         3        желудка, умерли, консультативно, бария, ана          3   
...     ...                                                ...        ...   
2579    197      замерзнет, зданиях, лях, отопительного, водой          2   
2580    199  накрывать, перерезает, лишает, логистические, ...          1   
2581    200                 пенальти, динамо, пол, лодзи, алли          1   
2582    202     панно, цветочное, склоне, печерским, удерживая          1   
2583    203       составы, участкам, салон, тревог, безопасных          1   

                   Timestamp  
0    2022-01-2

In [29]:
# MAKING EXCEL FILE FOR FUTURE PANEL DATA ANALYSIS

writer = pd.ExcelWriter('output_Agro_BERT.xlsx') 
topics_over_time.to_excel(writer) 
writer.save() 
from google.colab import files
files.download('output_Agro_BERT.xlsx')
print('DataFrame is written successfully to Excel File.')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

DataFrame is written successfully to Excel File.
